### Cleans files

In [1]:
import os
import pandas as pd

# Ask for the game ID once
game_id = input("Enter the game ID (e.g., 13e27d90): ").strip()

# Define base directory relative to the script's location
base_dir = os.path.join("..", "Data")

# Dynamically generate the list of file names using the game_id
csv_files = [
    f"{game_id}_summary.csv", 
    f"{game_id}_goalkeepers.csv",
    f"{game_id}_shots.csv",
    f"{game_id}_passing.csv",
    f"{game_id}_pass_types.csv", 
    f"{game_id}_defensive_actions.csv", 
    f"{game_id}_possession.csv", 
    f"{game_id}_misc_stats.csv"
]

def clean_dataframe(df):
    # Add game_id column
    df["game_id"] = game_id

    # Strip leading/trailing spaces from player column (if it exists)
    if "player" in df.columns:
        df["player"] = df["player"].astype(str).str.strip()

    # Create player_id column if both player and # exist
    if "player" in df.columns and "JerseyNumber" in df.columns:
        df["player_id"] = df["JerseyNumber"].astype(str).str.strip() + df["player"].astype(str) + df["game_id"].astype(str)

    return df

# Process each file
for file_name in csv_files:
    full_path = os.path.join(base_dir, file_name)
    if os.path.exists(full_path):
        df = pd.read_csv(full_path)
        cleaned_df = clean_dataframe(df)
        cleaned_df.to_csv(full_path, index=False)
        print(f"✅ Cleaned and saved: {file_name}")
    else:
        print(f"⚠️ File not found: {file_name}")


Enter the game ID (e.g., 13e27d90):  692f1ab8


✅ Cleaned and saved: 692f1ab8_summary.csv
✅ Cleaned and saved: 692f1ab8_goalkeepers.csv
✅ Cleaned and saved: 692f1ab8_shots.csv
✅ Cleaned and saved: 692f1ab8_passing.csv
✅ Cleaned and saved: 692f1ab8_pass_types.csv
✅ Cleaned and saved: 692f1ab8_defensive_actions.csv
✅ Cleaned and saved: 692f1ab8_possession.csv
✅ Cleaned and saved: 692f1ab8_misc_stats.csv


### MATCHES PLAYER ID

In [5]:
import pandas as pd
import os

# Ask for game ID
game_id = input("Enter the game ID (e.g., 13e27d90): ").strip()

# Define base directory relative to the script's location
base_dir = os.path.join("..", "Data")

# Load CSVs
df_summary = pd.read_csv(os.path.join(base_dir, f"{game_id}_summary.csv"))
df_shots = pd.read_csv(os.path.join(base_dir, f"{game_id}_shots.csv"))

# Determine correct player column names
summary_player_col = "player" if "player" in df_summary.columns else "player"
shots_player_col = "player" if "player" in df_shots.columns else "player"

# Create temp column for merging
df_summary["match_player"] = df_summary[summary_player_col]
df_shots["match_player"] = df_shots[shots_player_col]

# Merge on player name to bring in player_id
df_shots = df_shots.merge(
    df_summary[["match_player", "player_id"]],
    on="match_player",
    how="left"
)

# Append game_id to player_id for uniqueness
df_shots["player_id"] = df_shots["player_id"].astype(str) + game_id

# Clean up temp column
df_shots.drop(columns=["match_player"], inplace=True)

# Save updated file
df_shots.to_csv(os.path.join(base_dir, f"{game_id}_shots.csv"), index=False)
print(f"✅ Updated {game_id}_shots.csv with player_id")


Enter the game ID (e.g., 13e27d90):  692f1ab8


✅ Updated 692f1ab8_shots.csv with player_id


### PYTHON CODE TO ASSIST WITH SQL TABLE GEN

In [3]:
import pandas as pd
import os

# Ask for the game ID once
game_id = input("Enter the game ID (e.g., 13e27d90): ").strip()

# Path that goes up one level, then into /Data
data_folder = os.path.join("..", "Data")

filenames = [
    "match_metadata.csv",
    f"{game_id}_summary.csv", 
    f"{game_id}_goalkeepers.csv",
    f"{game_id}_shots.csv",
    f"{game_id}_passing.csv",
    f"{game_id}_pass_types.csv", 
    f"{game_id}_defensive_actions.csv", 
    f"{game_id}_possession.csv", 
    f"{game_id}_misc_stats.csv"
]

csv_files = [os.path.join(data_folder, name) for name in filenames]

for file in csv_files:
    if os.path.exists(file):
        df = pd.read_csv(file, encoding='cp1252')
        print(f"\n🗂️ Columns in {os.path.basename(file)}:")
        for col in df.columns:
            print(f"{col}")
    else:
        print(f"\n⚠️ File not found: {file}")


Enter the game ID (e.g., 13e27d90):  13e27d90



🗂️ Columns in match_metadata.csv:
game_id
match_date
match_time
attendance
home_team
away_team
home_score
away_score
possession_home
possession_away
passing_accuracy_home
passing_accuracy_away
shots_on_target_home
shots_on_target_away
saves_home
saves_away
fouls_home
fouls_away
corners_home
corners_away
crosses_home
crosses_away
touches_home
touches_away
tackles_home
tackles_away
interceptions_home
interceptions_away
aerials_won_home
aerials_won_away
clearances_home
clearances_away
offsides_home
offsides_away
goal_kicks_home
goal_kicks_away
throw_ins_home
throw_ins_away
long_balls_home
long_balls_away

🗂️ Columns in 13e27d90_summary.csv:
Player
JerseyNumber
Nation
Pos
Age
Min
Gls
Ast
PK
PKatt
Sh
SoT
CrdY
CrdR
Touches
Tkl
Int
Blocks
xG
npxG
xAG
SCA
GCA
Cmp
Att
CmpPct
PrgP
Carries
PrgC
Succ
game_id
player_id
team_name

🗂️ Columns in 13e27d90_goalkeepers.csv:
ï»¿Player
Nation
Age
Min
SoTA
GA
Saves
SavePct
PSxG
Cmp
PassAtt
CmpPct
GKAtt
Thr
LaunchPct
AvgLen
Opp
Stp
StpPct
OPA
AvgDist
Jerse